In [ ]:
!pip install sparse

In [ ]:
import h5py
import numpy as np
from Classes.ConvolutionLayer import Conv
from Classes.MaxpoolLayer import Maxpool
from Classes.FCLayer import FC
from Classes.SoftmaxLayer import Softmax
from Classes.ActivationLayer import Activation 
#import matplotlib.pyplot as plt
from Classes.MiscFunc import *
from Classes.DropoutLayer import Dropout
from PIL import Image
import os
import datetime

In [ ]:
!mkdir Accuracy
!mkdir Checkpoints

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
#Copy dataset from google drive
%cp -av /gdrive/My\ Drive/SortedResized /content/SortedResized

In [ ]:
def shuffleTrainingData():
  global usedPhotosDict
  global listOfAvailableClasses
  usedPhotosDict = {}
  listOfAvailableClasses = list(range(len(folders)))
  for folder in folders:
      trainSize = len(os.listdir(rootFolder+'/'+folder+'/train'))
      
      statDict = {}
      
      trainOrd = np.random.permutation(np.arange(trainSize))

      statDict["train"] = [0, trainSize, trainOrd]
      usedPhotosDict[folder] = statDict
  #print(usedPhotosDict)
  print("Images shuffled!")

def getRandomIndex(classInterval):
    maxim = classInterval[-1]
    
    randomNr = np.random.randint(maxim)
    #print(randomNr)
    for i in range(1,len(classInterval)):
        if(randomNr < classInterval[i] and randomNr > classInterval[i-1]):
            return i-1


def getNewBatch(batchType = "train"):
    global usedPhotosDict
    global trainClassIntervals
    global testClassIntervals
    global listOfAvailableClasses
    
    batch = np.zeros((batchSize, *imageSize))
    if(len(listOfAvailableClasses) != 0):  
        labels = [] 
        for i in range(batchSize):
      
            
            foundImg = 0
            while(foundImg == 0):
                if(batchType == "train"):
                    label = getRandomIndex(trainClassIntervals)
                elif(batchType == "test"):
                    pass
                    #label = getRandomIndex(testClassIntervals)
                else:
                    print("No such batch type!")
                if(label in listOfAvailableClasses):
                    foundImg = 1
            
            labels.append(label)
            folder = folders[label]
            indexes = usedPhotosDict[folder][batchType]
            currIndex, maxIndex, order = indexes 
            #print(folder, currIndex,"/",maxIndex)
            path = rootFolder + "/" + folder + "/" + batchType + "/" + str(order[currIndex]) + ".jpg"
            img = np.asarray(Image.open(path)) 

            batch[i] = np.transpose(img, axes = (2,1,0))

            usedPhotosDict[folder][batchType][0] = currIndex + 1 

            if(currIndex + 1 == maxIndex):
                #This class has no more available photos
                listOfAvailableClasses.remove(label)
        if(len(labels) == batchSize):
            labels = np.array(labels).astype(int)
            return batch, labels
        else:
            print("Not enough images to complete batch")
            return 0
    else:
        print("No more available "+batchType+"ing examples")
        return 0

def cnn_forward_test(img, label):
    out = conv1.forward(img)
    out = maxpool1.forward(out)
    out = relu1.forward(out)

    out = conv2.forward(out)
    out = maxpool2.forward(out)
    out = relu2.forward(out)
    out = out.reshape(batchSize,-1)

    out = fc1.forward(out)
    out = fc2.forward(out)
    out = softmax.forward(out)
    
    predicted = np.argmax(out,axis = 1)
    #print(label, predicted)
    acc = batchSize - np.count_nonzero(predicted - label)
        
    loss = crossEntropyLossForward(out, label)
    
    return acc, loss, out  

def cnn_forward(img, label):
    out = conv1.forward(img)
    out = drop1.forward(out)
    out = maxpool1.forward(out)
    out = relu1.forward(out)

    out = conv2.forward(out)
    out = drop2.forward(out)
    out = maxpool2.forward(out)
    out = relu2.forward(out)
    out = out.reshape(batchSize,-1)

    out = fc1.forward(out)
    out = drop3.forward(out)
    out = fc2.forward(out)
    out = softmax.forward(out)
    
    predicted = np.argmax(out,axis = 1)
    #print(label, predicted)
    acc = batchSize - np.count_nonzero(predicted - label)
        
    loss = crossEntropyLossForward(out, label)
    
    return acc, loss, out

 
def cnn_backprop(dLdOut, lr):
    out = softmax.backprop(dLdOut)    
    out = fc2.backprop(out, lr)
    out = drop3.backprop(out)
    out = fc1.backprop(out, lr).reshape(batchSize * 100, 1, 16, 22)
    out = relu2.backprop(out)
    out = maxpool2.backprop(out)
    out = drop2.backprop(out)
    out = conv2.backprop(out, lr)
    out = relu1.backprop(out)
    out = maxpool1.backprop(out)
    out = drop1.backprop(out)
    out = conv1.backprop(out, lr)

def saveNetwork(path):   
    with h5py.File(path,'w') as f:
        f.create_dataset('Conv1_W', data = conv1.W)
        f.create_dataset('Conv2_W', data = conv2.W)
        f.create_dataset('FC1_W', data = fc1.W)
        f.create_dataset('FC1_B', data = fc1.B)
        f.create_dataset('FC2_W', data = fc2.W)
        f.create_dataset('FC2_B', data = fc2.B)

def train(nameSave, errIndex, lr):
  print("Start Training")
  #lr = 0.005

  x = datetime.datetime.now()
  data = x.strftime("%d %B %H:%M") 
  print(data)

  lossFile = open("Accuracy/loss"+str(errIndex)+".txt","a")
  accFile = open("Accuracy/acc"+str(errIndex)+".txt","a")

  accFile.write("BatchSize = 5\n"+data+"\n") 

  shuffleTrainingData()

  finished = 0

  while(finished == 0):  
      returned = getNewBatch()

      if(returned != 0):
          batch, lb = returned
          accuracy, loss, out = cnn_forward(batch/255 - 0.5, lb)
          
          lossSum = np.sum(loss)

          lossFile.write(str(lossSum)+"\n")
          accFile.write(str(accuracy)+"\n")

          lossFile.flush()
          os.fsync(lossFile.fileno())

          accFile.flush()
          os.fsync(accFile.fileno())

          print(lossSum, accuracy)
          dLdOut = crossEntropyLossBackprop(out, lb)
          cnn_backprop(dLdOut, lr)
      else:
          finished = 1

  lossFile.close()
  accFile.close()

  saveNetwork("Checkpoints/" + nameSave)
  print("Finished Training")

testErrorDict = {}

def resetTestErrorDict():
  global testErrorDict
  testErrorDict = {}

  for folder in folders:
    nrImages = len(os.listdir(rootFolder+'/'+folder+'/test'))
    nrFullBatches = nrImages // batchSize

    maxImage = nrImages - (nrImages % nrFullBatches)


    testErrorDict[folder] = [0, maxImage]

  print(testErrorDict)


def getTestBatches():
  for lb, value in enumerate(testErrorDict.values()):
    maxImage = value[1]
    for batchNr in range(0, maxImage, batchSize):
      batch = np.zeros((batchSize, *imageSize))
      labels = []
      
      for i in range(batchSize):
        labels.append(lb)
        folder = folders[lb]
        path = rootFolder + "/" + folder + "/test/" + str(batchNr+i) + ".jpg"
        img = np.asarray(Image.open(path)) 
        batch[i] = np.transpose(img, axes = (2,1,0))
      yield batch, labels


#######################################
#test

def test(saveLogIndex):
  global nrOfRun
  lossFile1 = open("Accuracy/loss_test"+str(saveLogIndex)+".txt","a")
  accFile1 = open("Accuracy/acc_test"+str(saveLogIndex)+".txt","a")
  resetTestErrorDict()
  accFile1.write("BatchSize = 5\n")

  for batch, lb in getTestBatches():
      accuracy, loss, out = cnn_forward_test(batch/255 - 0.5, lb)
      
      testErrorDict[folders[lb[0] ]][0] += accuracy
      
      lossSum = np.sum(loss)

      print(lossSum, accuracy)

      lossFile1.write(str(lossSum)+"\n")
      accFile1.write(str(accuracy)+"\n")

      lossFile1.flush() 
      os.fsync(lossFile1.fileno())

      accFile1.flush()
      os.fsync(accFile1.fileno())

  for key in testErrorDict:
    accFile1.write(str(key)+ str(testErrorDict[key]))

  print(testErrorDict)

  lossFile1.close()
  accFile1.close()

def loadWeights(file):
    with h5py.File(file,'r') as f:
    #ls = list(f.keys())   
        conv1.W = np.array(f.get('Conv1_W')) 
        conv2.W = np.array(f.get('Conv2_W'))
        fc1.W = np.array(f.get('FC1_W'))
        fc1.B = np.array(f.get('FC1_B'))
        fc2.W = np.array(f.get('FC2_W'))
        fc2.B = np.array(f.get('FC2_B'))
    
    print("Weights loaded from", file)

In [ ]:
rootFolder = "SortedResized"
folders = os.listdir(rootFolder)
print(folders)
usedPhotosDict = {}
listOfAvailableClasses = []

shuffleTrainingData()

#------------------------------------------------------
trainClassIntervals = [0]

for value in usedPhotosDict.values():
    trainNr = value["train"][1]
    trainClassIntervals.append(trainClassIntervals[-1] + trainNr)
    
print(trainClassIntervals)

batchSize = 5
imageSize = (3, 144, 192)

##### Init layers
print("Initiate layers")
conv1 = Conv(10, (batchSize,3,144,192), batchSize=1, depth = 3,optimizer = 'momentum', filterSize = 6, stride = 2, isFirstLayer = 1)
#output = (batchSize*10, 1, 70, 94)
drop1 = Dropout(0.9)
maxpool1 = Maxpool()
#output = (batchSize*10, 1, 44, 59)
relu1 = Activation(activation = 'leaky_relu')

conv2 = Conv(10, (batchSize * 10, 1, 35, 47), batchSize=1, depth = 1,optimizer = 'momentum', filterSize = 4, stride = 1, isFirstLayer = 0)
#output = (batchSize * 100, 1, 32, 44)
drop2 = Dropout(0.9)
maxpool2 = Maxpool()
relu2 = Activation(activation = 'leaky_relu')
#output = (batchSize * 100, 1, 16, 22)
# flatten size = batchSize * 100 * 16 *22 = 176000 = batchSize * 35200
fc1 = FC(35200, 100, batchSize=batchSize,optimizer = 'momentum')
drop3 = Dropout(0.5)
fc2 = FC(100, 5, batchSize=batchSize,optimizer = 'momentum')
softmax = Softmax()
print("Layer initialization done")


loadWeights("CNN_9_July9_acc82.h5")




['Left', 'NoAttention', 'Front', 'Phone', 'Right']
Images shuffled!
[0, 275, 604, 914, 1321, 1565]
Initiate layers
Layer initialization done
Weights loaded from CNN_9_July9_acc82.h5


In [ ]:
lr = 0.0015
i=0
train("CNN_10_lr_0015_.h5", 18+i)
test(9+i)

Start Training
10 July 17:17
Images shuffled!
0.011928435756969944 5
0.0902162596215424 5
0.006343438617545647 5
0.005928468811095306 5
0.028938391609465527 5
0.9725718792597835 4
0.09388481689585183 5
0.007727703579710677 5
0.01593870045159007 5
0.04702581590496473 5
0.00565247255590013 5
4.070898829143249 3
0.027784184116702998 5
0.04649119017537147 5
0.059805286186792705 5
0.005737448331583178 5
0.0017084132773942456 5
0.005174324269026342 5
0.07712913166793722 5
0.1731219455209907 5
0.23402414455256937 5
0.17393450435358807 5
0.02854338202355072 5
0.0451643678787888 5
0.0617143717403648 5
0.0019007934549351453 5
0.007861753030682235 5
0.29606972244646446 5
0.07140691610823884 5
0.8587560518577344 4
0.001295395304841438 5
0.021711586628817288 5
0.08624488445715671 5
0.003621378881807525 5
0.007337355047288355 5
0.4067388595878573 5
0.03773067912765009 5
0.13182088860995167 5
0.4817698664713369 5
0.31626249426997816 5
0.000574046003129879 5
0.1680107388576629 5
0.007529962881467474 5

In [ ]:

#Check filter size and stride
y,x = (35, 47)
for filterSize in range(3,20):
    for stride in range(1,6):
        print(filterSize, stride, (y - filterSize) / stride + 1, (x - filterSize) / stride + 1)

In [ ]:


def 